In [2]:
import numpy as np
import pandas as pd
import os
import time
import copy

from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data as data_utils
from torchvision import datasets, models, transforms

from sklearn.model_selection import KFold, train_test_split

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from tqdm import tqdm
import Models

# Connect your script to Neptune
import neptune

In [3]:
# Prefix data directory
prefix_dir = '.'

env = 'lk3'

# Use Yolo
use_yolo = True
cropped = 'cropped2_' if use_yolo else ''

# Top level data directory. Here we assume the format of the directory conforms
# to the ImageFolder structure
train_dir = f'{prefix_dir}/data/{cropped}train_imgs'

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = 'resnext50_32x4d'

# Number of classes in the dataset
num_classes = 48

# Batch size for training (change depending on how much memory you have)
batch_size = 64

# Number of epochs and earlystop to train for
num_epochs = 200

# validation set ratio
num_splits = 10
num_earlystop = 20 if num_epochs // 10 < 20 else num_epochs // 10
# not use
num_earlystop = 0

# Iput size for resize imgae
input_size = 128

# Learning rate for optimizer
learning_rate = 0.01

# Use K-folds
use_kfolds = False

# Use multi-GPU
cuda_num = 2

In [4]:
df = pd.read_csv(f'{prefix_dir}/data/pre-processed/{cropped}train_df.csv')

imgs = df.iloc[:, 0].to_numpy()
motions = df.iloc[:, 1:]
columns = motions.columns.to_list()[::2]
class_labels = [label.replace('_x', '').replace('_y', '') for label in columns]
keypoints = []
for motion in motions.to_numpy():
    a_keypoints = []
    for i in range(0, motion.shape[0], 2):
        a_keypoints.append((float(motion[i]), float(motion[i+1])))
    keypoints.append(a_keypoints)
keypoints = np.array(keypoints)

---

In [22]:
imgs[0]

'001-1-1-01-Z17_A-0000001.jpg'

In [21]:
"".join(imgs[0].split("-")[:-1])

'0011101Z17_A'

In [29]:
np.where(imgs == imgs[1])

(array([1], dtype=int64),)

In [6]:
def train_val_split(imgs, keypoints, random_state):
    d = dict()
    for file in imgs:
        key = "".join(file.split("-")[:-1])
        if key not in d.keys():
            d[key] = [file]
        else:
            d[key].append(file)

    np.random.seed(random_state)
    trains = []
    validations = []
    for key, value in d.items():
        r = np.random.randint(len(value), size=2)
        for i in range(len(value)):
            if i in r:
                validations.append(np.where(imgs == value[i])[0][0])
            else:
                trains.append(np.where(imgs == value[i])[0][0])
    return imgs[trains], imgs[validations], keypoints[trains], keypoints[validations]

In [7]:
train_imgs, valid_imgs, train_keypoints, valid_keypoints = train_val_split(imgs, keypoints, random_state=42)

In [16]:
train_imgs

array(['001-1-1-01-Z17_A-0000001.jpg', '001-1-1-01-Z17_A-0000003.jpg',
       '001-1-1-01-Z17_A-0000005.jpg', ...,
       '642-2-4-31-Z148_E-0000027.jpg', '642-2-4-31-Z148_E-0000029.jpg',
       '642-2-4-31-Z148_E-0000031.jpg'], dtype=object)

In [13]:
train_imgs.shape

(3679,)

In [14]:
valid_imgs.shape

(516,)

In [12]:
train_keypoints.shape

(3679, 24, 2)

In [15]:
valid_keypoints.shape

(516, 24, 2)

---

In [4]:
ns = neptune.init(project_qualified_name='mybirth0407/dacon-motion',
             api_token=neptune_config.token)

# Create experiment
neptune.create_experiment(f'{model_name}')

neptune.log_metric('batch_size', batch_size)
neptune.log_metric('num_epochs', num_epochs)
neptune.log_metric('num_splits', num_splits)
neptune.log_metric('num_ealrystop', num_earlystop)
neptune.log_metric('input_size', input_size)
neptune.log_metric('learning_rate', learning_rate)
neptune.log_metric('use_kfolds', use_kfolds)
neptune.log_metric('use_yolo', use_yolo)

https://ui.neptune.ai/mybirth0407/dacon-motion/e/DAC-212


In [5]:
counter = ns._get_current_experiment()._id
os.mkdir(f'{prefix_dir}/{env}/{counter}')
print(counter)

DAC-212


In [6]:
def train_model(model, dataloaders, criterion, optimizer, earlystop=0, num_epochs=25, monitor='val', allsave=False, phases=['train', 'val']):
    since = time.time()
    
    train_loss_history = []
    val_loss_history = []
    
    earlystop_value = 0

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 999999999
    
    for epoch in range(num_epochs):
        epoch_since = time.time()
        if earlystop and earlystop_value >= earlystop:
            break

        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in phases:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs.float(), labels.float())

                    # for classification
#                     _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                # for classification
#                 running_corrects += torch.sum(preds == labels.data)
                # for regression
#                 running_corrects += torch.sum(outputs.float() == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
#             epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            epoch_time_elapsed = time.time() - epoch_since
            print('{} ({}) Loss: {:.4f} Elapsed time: {:.0f}m {:.0f}s'.format(
                phase, len(dataloaders[phase].dataset), epoch_loss, epoch_time_elapsed // 60, epoch_time_elapsed % 60))
            neptune.log_metric(f'{phase}_loss', epoch_loss)
#             neptune.log_metric(f'{phase}_acc', epoch_acc)
            
                
            # deep copy the model
            if phase == 'val':
                if monitor == 'val':
                    if epoch_loss < best_loss:
                        best_loss = epoch_loss
                        neptune.log_metric(f'{phase}_best_loss', best_loss)
                        best_model_wts = copy.deepcopy(model.state_dict())
                        torch.save(model_ft.state_dict(), f'{prefix_dir}/{env}/{counter}/{model_name}.pt')
                        print('copied model')
                        earlystop_value = 0
                    else:
                        earlystop_value += 1
                        if allsave:
                            torch.save(model_ft.state_dict(), f'{prefix_dir}/{env}/{counter}/{model_name}_{epoch_loss:.2f}_{epoch}.pt')
                    val_loss_history.append(epoch_loss)
            elif phase == 'train':
                if monitor == 'train':
                    if epoch_loss < best_loss:
                        best_loss = epoch_loss
                        neptune.log_metric(f'{phase}_best_loss', best_loss)
                        best_model_wts = copy.deepcopy(model.state_dict())
                        torch.save(model_ft.state_dict(), f'{prefix_dir}/{env}/{counter}/{model_name}.pt')
                        print('copied model')
                        earlystop_value = 0
                    else:
                        earlystop_value += 1
                        if allsave:
                            torch.save(model_ft.state_dict(), f'{prefix_dir}/{env}/{counter}/{model_name}_{epoch_loss:.2f}_{epoch}.pt')
                    train_loss_history.append(epoch_loss)
        print()

    time_elapsed = time.time() - since
    print('Training and Validation complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    if monitor == 'val':
        print('Best Validation Loss: {:4f}\n'.format(best_loss))
    elif monitor == 'train':
        print('Best Training Loss: {:4f}\n'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_loss

In [7]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [8]:
# # Initialize the model for this run
model_ft, input_size = model.initialize_model(model_name, input_size, num_classes, use_pretrained=True)
# set_parameter_requires_grad(model_ft, feature_extract)

# Detect if we have a GPU available
device = torch.device(f'cuda:{cuda_num}' if torch.cuda.is_available() else 'cpu')

# Send the model to GPU
model_ft = model_ft.to(device)

# Multi GPU
# os.environ["CUDA_VISIBLE_DEVICES"] = '2, 3'
model = nn.DataParallel(model_ft, device_ids=[2, 3], output_device=2)

# Print the model we just instantiated
# print(model_ft)

In [9]:
# # Data augmentation and normalization for training
# # Just resize and normalization for validation

A_transforms = {
    'train':
        A.Compose([
            A.Resize(input_size, input_size, always_apply=True),
            A.RandomBrightnessContrast(p=0.3),
#             A.HorizontalFlip(p=0.3),
            A.OneOf([
                A.RandomRotate90(p=1),
                A.VerticalFlip(p=1),
            ], p=0.5),
            A.OneOf([
                A.MotionBlur(p=1),
                A.GaussNoise(p=1)                 
            ], p=0.5),
            
            A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ToTensorV2()
        ], keypoint_params=A.KeypointParams(format='xy', label_fields=['class_labels'], remove_invisible=False, angle_in_degrees=True)),
    
    'val':
        A.Compose([
            A.Resize(input_size, input_size, always_apply=True),
            A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ToTensorV2()
        ], keypoint_params=A.KeypointParams(format='xy', label_fields=['class_labels'], remove_invisible=False, angle_in_degrees=True)),
    
    'test':
        A.Compose([
            A.Resize(input_size, input_size, always_apply=True),
            A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
}

In [10]:
class Dataset(data_utils.Dataset):
    """__init__ and __len__ functions are the same as in TorchvisionDataset"""
    def __init__(self, data_dir, imgs, keypoints, phase, class_labels=None, data_transforms=None):
        self.data_dir = data_dir
        self.imgs = imgs
        self.keypoints = keypoints
        self.phase = phase
        self.class_labels = class_labels
        self.data_transforms = data_transforms

    def __getitem__(self, idx):
        # Read an image with OpenCV
        img = cv2.imread(os.path.join(self.data_dir, self.imgs[idx]))
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        keypoints = self.keypoints[idx]
    
        if self.data_transforms:
            augmented = self.data_transforms[self.phase](image=img, keypoints=keypoints, class_labels=self.class_labels)
            img = augmented['image']
            keypoints = augmented['keypoints']
        keypoints = np.array(keypoints).flatten()

        return img, keypoints
    
    def __len__(self):
        return len(self.imgs)

In [11]:
# Setup the loss fxn
criterion = nn.MSELoss()

print(f'k-folds use: {use_kfolds}')
print(f'yolo use: {use_yolo}')

full_since = time.time()

if use_kfolds:
    kf = KFold(num_splits, random_state=42, shuffle=True)

    for i, (train_index, val_index) in enumerate(kf.split(imgs)):
        print(f'{i+1}/{num_splits} folds iteration')
        since = time.time()
        X_train, X_val = imgs[train_index], imgs[val_index]
        y_train, y_val = keypoints[train_index], keypoints[val_index]
        train_data = Dataset(train_dir, X_train, y_train, data_transforms=A_transforms, class_labels=class_labels, phase='train')
        val_data = Dataset(train_dir, X_val, y_val, data_transforms=A_transforms, class_labels=class_labels, phase='val')
        train_loader = data_utils.DataLoader(train_data, batch_size=batch_size, shuffle=True)
        val_loader = data_utils.DataLoader(val_data, batch_size=batch_size, shuffle=False)
        dataloaders = {'train': train_loader, 'val': val_loader}

        # Observe that all parameters are being optimized
        optimizer_ft = optim.Adam(model_ft.parameters(), lr=learning_rate)

        # Train and evaluate
        model_ft, best_loss = train_model(
            model_ft, dataloaders, criterion, optimizer_ft,
            num_epochs=num_epochs, earlystop=num_earlystop)
        torch.save(model_ft.state_dict(), f'{prefix_dir}/{env}/{counter}/{model_name}_{i+1}_{best_loss:.2f}.pt')
        time_elapsed = time.time() - since
        print('Elapsed time: {:.0f}m {:.0f}s\n'.format(time_elapsed // 60, time_elapsed % 60))
else:
    since = time.time()
    X_train, X_val, y_train, y_val = train_test_split(imgs, keypoints, test_size=1/num_splits, random_state=42)
    train_data = Dataset(train_dir, X_train, y_train, data_transforms=A_transforms, class_labels=class_labels, phase='train')
    val_data = Dataset(train_dir, X_val, y_val, data_transforms=A_transforms, class_labels=class_labels, phase='val')
    train_loader = data_utils.DataLoader(train_data, batch_size=batch_size, num_workers=8, shuffle=True)
    val_loader = data_utils.DataLoader(val_data, batch_size=batch_size, num_workers=8, shuffle=False)
    dataloaders = {'train': train_loader, 'val': val_loader}

    # Observe that all parameters are being optimized
    optimizer_ft = optim.Adam(model_ft.parameters(), lr=learning_rate)

    # Train and evaluate
    model_ft, best_loss = train_model(
        model_ft, dataloaders, criterion, optimizer_ft,
        num_epochs=num_epochs, earlystop=num_earlystop)
    torch.save(model_ft.state_dict(), f'{prefix_dir}/{env}/{counter}/{model_name}_{best_loss:.2f}.pt')
    time_elapsed = time.time() - since
    print('Elapsed time: {:.0f}m {:.0f}s\n'.format(time_elapsed // 60, time_elapsed % 60))

fulltime_elapsed = time.time() - full_since
print('All process done!\nElapsed time: {:.0f}m {:.0f}s\n'.format(fulltime_elapsed // 60, fulltime_elapsed % 60))

k-folds use: False
yolo use: True
Epoch 1/200
----------
train (3775) Loss: 892.9218 Elapsed time: 0m 21s
val (420) Loss: 18581.3934 Elapsed time: 0m 23s
copied model

Epoch 2/200
----------
train (3775) Loss: 599.3248 Elapsed time: 0m 21s
val (420) Loss: 482.8133 Elapsed time: 0m 23s
copied model

Epoch 3/200
----------
train (3775) Loss: 433.3855 Elapsed time: 0m 21s
val (420) Loss: 300.7220 Elapsed time: 0m 23s
copied model

Epoch 4/200
----------
train (3775) Loss: 336.1645 Elapsed time: 0m 21s
val (420) Loss: 189.1350 Elapsed time: 0m 23s
copied model

Epoch 5/200
----------
train (3775) Loss: 282.6212 Elapsed time: 0m 21s
val (420) Loss: 414.8952 Elapsed time: 0m 23s

Epoch 6/200
----------
train (3775) Loss: 246.4367 Elapsed time: 0m 21s
val (420) Loss: 204.6039 Elapsed time: 0m 23s

Epoch 7/200
----------
train (3775) Loss: 191.0396 Elapsed time: 0m 21s
val (420) Loss: 123.8226 Elapsed time: 0m 23s
copied model

Epoch 8/200
----------
train (3775) Loss: 167.8664 Elapsed time: 0

In [12]:
# model_ft.load_state_dict(torch.load(f'{prefix_dir}/{env}/{counter}/resnet18_18.95.pt'))

In [13]:
test_dir = f'./data/{cropped}test_imgs'
test_imgs = os.listdir(test_dir)
test_imgs.sort()

In [14]:
class TestDataset(data_utils.Dataset):
    """__init__ and __len__ functions are the same as in TorchvisionDataset"""
    def __init__(self, data_dir, imgs, phase, data_transforms=None):
        self.data_dir = data_dir
        self.imgs = imgs
        self.phase = phase
        self.data_transforms = data_transforms

    def __getitem__(self, idx):
        filename = self.imgs[idx]
        # Read an image with OpenCV
        img = cv2.imread(os.path.join(self.data_dir, self.imgs[idx]))
        h = img.shape[0]
        w = img.shape[1]
        if self.data_transforms:
            augmented = self.data_transforms[self.phase](image=img)
            img = augmented['image']
        return filename, img, (h, w)
    
    def __len__(self):
        return len(self.imgs)
    
test_data = TestDataset(test_dir, test_imgs, data_transforms=A_transforms, phase='test')
test_loader = data_utils.DataLoader(test_data, batch_size=batch_size * 4, shuffle=False)

In [15]:
all_predictions = []
files = []
shapes = []
with torch.no_grad():
    for filenames, inputs, shape in tqdm(test_loader):
        predictions = list(model_ft(inputs.to(device)).cpu().numpy())
        files.extend(filenames)
        
        shapes.extend(shape)
        for prediction in predictions:
            all_predictions.append(prediction)
            
origin_shape_y = shapes[0].numpy()
origin_shape_x = shapes[1].numpy()
for i in range(1, len(shapes) // 2):
    origin_shape_y = np.append(origin_shape_y, shapes[2*i].numpy())
    origin_shape_x = np.append(origin_shape_x, shapes[2*i + 1].numpy())

all_predictions = np.array(all_predictions)
for i in range(all_predictions.shape[0]):
    all_predictions[i, [2*j for j in range(num_classes//2)]] /= input_size / origin_shape_x[i]
    all_predictions[i, [2*j + 1 for j in range(num_classes//2)]] /= input_size / origin_shape_y[i]

100%|██████████| 7/7 [00:15<00:00,  2.19s/it]


In [16]:
res_df = pd.read_csv(f'{prefix_dir}/data/res2_test_df.csv')
res = res_df.iloc[:, 1:].to_numpy()

all_predictions = np.array(all_predictions)
for i in range(all_predictions.shape[0]):
    all_predictions[i, [2*j for j in range(num_classes//2)]] += res[i][0]
    all_predictions[i, [2*j + 1 for j in range(num_classes//2)]] += res[i][1]

In [17]:
df_sub = pd.read_csv(f'{prefix_dir}/data/sample_submission.csv')
df = pd.DataFrame(columns=df_sub.columns)
df['image'] = files
df.iloc[:, 1:] = all_predictions
df.head()

,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,...,right_palm_x,right_palm_y,spine2(back)_x,spine2(back)_y,spine1(waist)_x,spine1(waist)_y,left_instep_x,left_instep_y,right_instep_x,right_instep_y
0,649-2-4-32-Z148_A-0000001.jpg,1089.400146,526.3927,1085.796021,547.372009,1106.544189,543.517212,1054.99707,591.559448,1123.040039,...,1055.435547,284.209351,955.68866,515.05072,886.791382,485.79895,694.466431,690.139221,831.733643,649.395386
1,649-2-4-32-Z148_A-0000003.jpg,1091.523926,526.389648,1087.561279,547.47644,1108.689453,543.553467,1055.715576,590.705017,1124.900635,...,1058.001709,286.759888,955.491089,513.697632,886.659912,484.183716,692.194824,689.83429,832.298584,649.864136
2,649-2-4-32-Z148_A-0000005.jpg,1100.746216,520.331299,1098.3479,540.6651,1119.109863,536.957764,1066.727783,584.766602,1139.15686,...,1087.047974,281.664551,969.650391,524.667603,897.210571,500.809143,691.565125,685.203613,839.131531,645.434814
3,649-2-4-32-Z148_A-0000007.jpg,1173.5802,541.876221,1179.048462,555.010498,1183.324219,547.131714,1159.481689,592.131714,1173.720215,...,1026.306641,521.795715,1014.225708,579.08667,936.80072,566.228638,705.465698,693.574524,741.414185,636.273438
4,649-2-4-32-Z148_A-0000009.jpg,1112.44519,509.738739,1110.798096,531.206665,1130.710693,525.843262,1081.217896,580.119263,1152.450562,...,1144.221924,295.797913,982.062256,532.376587,909.259644,516.29187,684.78064,699.79718,827.391541,657.276123


In [18]:
df.to_csv(f'{prefix_dir}/submissions/{counter}_{model_name}_{best_loss:.2f}.csv', index=False)

In [19]:
# tune_train_data = Dataset(train_dir, X_val, y_val, data_transforms=A_transforms, class_labels=class_labels, phase='train')
# tune_val_data = Dataset(train_dir, X_train, y_train, data_transforms=A_transforms, class_labels=class_labels, phase='val')
# tune_train_loader = data_utils.DataLoader(tune_train_data, batch_size=batch_size, shuffle=True)
# tune_val_loader = data_utils.DataLoader(tune_val_data, batch_size=batch_size, shuffle=False)
# tune_dataloaders = {'train': tune_train_loader, 'val': tune_val_loader}

# # Observe that all parameters are being optimized
# optimizer_ft = optim.Adam(model_ft.parameters(), lr=learning_rate)

# # Train and evaluate
# since = time.time()
# model_ft, best_loss = train_model(
#     model_ft, tune_dataloaders, criterion, optimizer_ft,
#     num_epochs=10, earlystop=0, allsave=True)
# torch.save(model_ft.state_dict(), f'{prefix_dir}/{env}/{counter}/{model_name}_tuned_{best_loss:.2f}.pt')
# time_elapsed = time.time() - since
# print('Elapsed time: {:.0f}m {:.0f}s\n'.format(time_elapsed // 60, time_elapsed % 60))

In [21]:
tune_train_data = Dataset(train_dir, imgs, keypoints, data_transforms=A_transforms, class_labels=class_labels, phase='train')
tune_train_loader = data_utils.DataLoader(tune_train_data, batch_size=batch_size, shuffle=True)
tune_dataloaders = {'train': tune_train_loader}

# Observe that all parmeters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=learning_rate)

# Train and evaluate
since = time.time()
model_ft, best_loss = train_model(
    model_ft, tune_dataloaders, criterion, optimizer_ft,
    num_epochs=3, earlystop=0, allsave=True, monitor='train', phases=['train'])
torch.save(model_ft.state_dict(), f'{prefix_dir}/{env}/{counter}/{model_name}_tuned_{best_loss:.2f}.pt')
time_elapsed = time.time() - since
print('Elapsed time: {:.0f}m {:.0f}s\n'.format(time_elapsed // 60, time_elapsed % 60))

Epoch 1/3
----------
train (4195) Loss: 8.8942 Elapsed time: 1m 36s
copied model

Epoch 2/3
----------
train (4195) Loss: 8.4387 Elapsed time: 1m 35s
copied model

Epoch 3/3
----------
train (4195) Loss: 8.3217 Elapsed time: 1m 35s
copied model

Training and Validation complete in 4m 47s
Best Training Loss: 8.321739

Elapsed time: 4m 47s



In [22]:
all_predictions = []
files = []
shapes = []
with torch.no_grad():
    for filenames, inputs, shape in tqdm(test_loader):
        predictions = list(model_ft(inputs.to(device)).cpu().numpy())
        files.extend(filenames)
        
        shapes.extend(shape)
        for prediction in predictions:
            all_predictions.append(prediction)
            
origin_shape_y = shapes[0].numpy()
origin_shape_x = shapes[1].numpy()
for i in range(1, len(shapes) // 2):
    origin_shape_y = np.append(origin_shape_y, shapes[2*i].numpy())
    origin_shape_x = np.append(origin_shape_x, shapes[2*i + 1].numpy())

100%|██████████| 7/7 [00:24<00:00,  3.46s/it]


In [23]:
all_predictions = np.array(all_predictions)
for i in range(all_predictions.shape[0]):
    all_predictions[i, [2*j for j in range(num_classes//2)]] /= input_size / origin_shape_x[i]
    all_predictions[i, [2*j + 1 for j in range(num_classes//2)]] /= input_size / origin_shape_y[i]
    
res_df = pd.read_csv(f'{prefix_dir}/data/res2_test_df.csv')
res = res_df.iloc[:, 1:].to_numpy()

all_predictions = np.array(all_predictions)
for i in range(all_predictions.shape[0]):
    all_predictions[i, [2*j for j in range(num_classes//2)]] += res[i][0]
    all_predictions[i, [2*j + 1 for j in range(num_classes//2)]] += res[i][1]

In [24]:
df_sub = pd.read_csv(f'{prefix_dir}/data/sample_submission.csv')
df = pd.DataFrame(columns=df_sub.columns)
df['image'] = files
df.iloc[:, 1:] = all_predictions
df.head()

,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,...,right_palm_x,right_palm_y,spine2(back)_x,spine2(back)_y,spine1(waist)_x,spine1(waist)_y,left_instep_x,left_instep_y,right_instep_x,right_instep_y
0,649-2-4-32-Z148_A-0000001.jpg,1129.631104,522.625,1131.156006,527.815735,1141.35144,523.064392,1112.727051,544.559082,1138.172241,...,1070.983765,414.300049,996.789185,564.46228,911.210815,561.5979,723.179443,574.711853,787.485657,561.435791
1,649-2-4-32-Z148_A-0000003.jpg,1131.684937,523.540344,1133.579102,528.741943,1143.230835,523.817749,1114.635742,545.254272,1138.894043,...,1072.407349,414.599213,996.275146,564.355591,909.294861,561.802063,724.853271,565.891479,785.125732,554.425049
2,649-2-4-32-Z148_A-0000005.jpg,1157.23291,519.015991,1159.444092,525.924316,1168.587036,519.792847,1135.520508,547.313599,1157.648315,...,1081.33374,411.499146,1022.998047,578.993042,936.562073,583.102539,712.629761,573.341614,784.264038,549.679993
3,649-2-4-32-Z148_A-0000007.jpg,1207.16626,618.785217,1218.654297,615.824829,1216.127075,608.076538,1204.846924,611.371887,1201.344849,...,1129.730103,644.426025,1074.061523,632.277466,979.594849,623.125122,752.833496,550.59845,698.846436,543.408569
4,649-2-4-32-Z148_A-0000009.jpg,1169.14209,522.656494,1171.640259,531.883301,1186.068848,522.871887,1144.223633,553.479736,1182.264404,...,1183.747925,432.206146,1031.474243,570.272461,938.329834,579.658813,698.909668,596.77124,787.199219,556.950928


In [25]:
df.to_csv(f'{prefix_dir}/submissions/{counter}_{model_name}_tuned_{best_loss:.2f}.csv', index=False)

In [26]:
neptune.stop()
torch.cuda.empty_cache()

In [27]:
print(counter)

DAC-212
